# EDA

## Load Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [15]:
from datetime import datetime
# 전처리 코드
def preprocessing():
    df = pd.read_csv("./socar_reservation_triplog.csv")
    df["reservation_return_at"] = pd.to_datetime(df["reservation_return_at"])
    df["reservation_start_at"] = pd.to_datetime(df["reservation_start_at"])
    
    # 시간에러 데이터 삭제 (빌린시간과 반납시간 차이 에러)
    df["total_rent_time"] = (df["reservation_return_at"] - df["reservation_start_at"]).dt.total_seconds()
    negative_rent_time_index = df[df['total_rent_time'] <=0 ].index
    df = df.drop(negative_rent_time_index)
    
    # 이동 거리가 0인 사람 삭제 => 한번만 사용한 사람들이 모두 여기에 해당하는 사람들은 아니므로 삭제
    # - 겹치는 id를 가진 고객이라도 member_total_distance는 동일
    #   - 즉, 이용 즉시 갱신되어서 모두 같은 값을 가지고 있다는 것
    #   - 그러므로 member_total_distance가 0인 고객들은 처음 사용한 고객 혹은 실질적으로 운전하지 않은 사람들의 데이터라고 볼 수 있음
    #   - 보통 거리가 시간이 아주 짧음
    # - 1회당 주행한 거리를 나타내는 지표는 현재 없음
    zero_distance = df.loc[df["member_total_distance"] == 0].index
    df = df.drop(zero_distance)
    
    # - 예약한 곳의 위도나 경도는 크게 중요하지 않을 듯 그러므로 삭제해도 될 듯
    # - 차량 사용 시작시간(reservation_start_at)과 예약완료 시간이 있었다면 오히려 사용성이 높았을듯
    #     - 예를들면, 급하게 예약하고 사용했다는 것이니까 위치가 중요할듯 (행사가 있는 곳, 혹은 유동인구가 많은곳으로 추정할 수 있음, 또한 연령대와 같이 비교해서 어떤 연령이 예약직후 바로 사용하는지도 확인 가능 할 듯)
    df = df.drop(columns=["reservation_created_lat", "reservation_created_lng"])


In [44]:
ddf = pd.read_csv("./socar_reservation_triplog.csv")
ddf.describe()

,reservation_id,car_id,member_age,member_total_distance,zone_lat,zone_lng,reservation_created_lat,reservation_created_lng
count,7.515480e+05,751548.000000,751548.000000,738010.000000,751548.000000,751548.000000,199701.000000,199701.000000
mean,2.395360e+07,11032.632738,30.747854,4706.664131,37.286415,127.167624,37.263305,127.134576
std,2.155359e+06,3979.120987,8.424734,6908.222464,0.638582,0.609597,0.880109,1.513531
min,1.960042e+07,4169.000000,21.000000,0.000000,35.083052,126.597065,-42.968955,-123.128056
25%,2.207456e+07,7833.000000,25.000000,750.000000,37.395187,126.883526,37.391814,126.901675
50%,2.396182e+07,10644.000000,28.000000,2302.000000,37.520612,127.041004,37.516770,127.038024
75%,2.584444e+07,14507.000000,35.000000,5923.000000,37.590836,127.122589,37.588336,127.121718
max,2.762850e+07,19449.000000,85.000000,121163.000000,37.760854,129.432836,49.277712,151.179826


## EDA 주요 내용
#### 데이터 전처리
- 시간이상치 삭제
- 주행거리가 0인 사람 삭제
- 필요 없는 컬럼 삭제(reservation_created_lat, reservation_created_lng)

## 추가 작업
- 지역별로 가장 많이 사용한 차종 count
  - `get_region_data()`
  - `get_car_count()`
  - `most_popular_car()`
- 고객별 사용한 빈도 파악
- 부름서비스를 이용한 고객 파악
  - 2번이상 사용한 고객

## 외부 데이터
- 유동인구 데이터
  > https://data.gg.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=6NOSUTB9NJAKABX5DFCJ31197470&infSeq=1&order=

  > https://bigdata.seoul.go.kr/data/selectPageListSampleDataSet.do?r_id=P213&sample_data_seq=329&tab_type=&sch_cate=10&file_id=&sch_text=&sch_order=U&currentPage=2
- 상권 데이터
  > https://data.seoul.go.kr/dataList/OA-15568/S/1/datasetView.do

In [16]:
# 차량 종류 카운트
def get_car_count():
    df = pd.read_csv("./socar_reservation_triplog.csv")
    # car_name_data= df.groupby(['car_name']).count()
    car_name_data = df['car_name'].value_counts()
    print(car_name_data)
    for i in car_name_data:
        print(i)
get_car_count()

아반떼AD             126226
코나                 90322
더뉴레이               85058
넥스트스파크             49472
레이                 46102
                   ...  
휠체어슬로프 X 카니발          17
LF쏘나타                 15
스타렉스                   9
[쏘카세이브] 투싼(경유)         2
볼트EV                   1
Name: car_name, Length: 61, dtype: int64
126226
90322
85058
49472
46102
41091
35227
30234
29115
25293
18243
17406
15743
12966
12050
10803
9148
8276
7884
7381
6350
5605
5025
4540
4348
4026
3651
3593
3466
3302
3081
3064
2801
2657
2170
1756
1646
1644
1640
1553
1239
1230
1053
1024
871
660
498
294
228
111
93
84
59
57
28
17
17
15
9
2
1


In [18]:
# 존 유형 카운트
def get_zone_count():
    df = pd.read_csv("./socar_reservation_triplog.csv")
    unique_zone = df['zone_type1'].value_counts()
    print(unique_zone)

get_zone_count()

LIVING_APT               280614
TRANSFER_SUBWAY          146351
LIVING_ETC               105175
LIVING_VILLA              88749
COMMERCIAL_HOTSPOT        33069
TRANSFER_STATION          32935
COMMERCIAL_OFFICE         15917
TRANSFER_TERMINAL         14193
SCHOOL_OUT                 6468
TRANSFER_BUS               5237
COMMERCIAL_LODGE           2739
SCHOOL_IN                  1264
COMMERCIAL_INDUSTRIAL       778
ETC                          19
Name: zone_type1, dtype: int64


In [29]:
# 가장 많이 사용한 지역 추출
def get_region_data():
    df = pd.read_csv("./socar_reservation_triplog.csv")
    unique_region = df['region'].value_counts(ascending=False)
    print(unique_region)
    region_list_result = unique_region.index.tolist()
    print(region_list_result)
    
    return region_list_result


region_list = get_region_data()

서울특별시 강남구       98915
서울특별시 강서구       77535
경기도 성남시 분당구     46782
서울특별시 성북구       41192
서울특별시 서대문구      39427
서울특별시 동대문구      37412
경기도 화성시         35249
경기도 고양시 일산동구    31906
경기도 남양주시        26546
경기도 고양시 덕양구     26253
경기도 의정부시        25225
경기도 안양시 동안구     22918
경기도 광명시         22645
울산광역시 남구        21836
경기도 김포시         21500
경기도 성남시 수정구     18773
경기도 안양시 만안구     16806
경기도 성남시 중원구     16238
경기도 고양시 일산서구    16052
세종특별자치시 세종시     15263
전라북도 전주시 완산구    15013
울산광역시 울주군       13632
경기도 하남시         13456
경기도 구리시         13011
전라북도 전주시 덕진구    12707
부산광역시 강서구       10634
울산광역시 동구         4526
울산광역시 북구         3799
울산광역시 중구         3310
경기도 양평군          2987
Name: region, dtype: int64
['서울특별시 강남구', '서울특별시 강서구', '경기도 성남시 분당구', '서울특별시 성북구', '서울특별시 서대문구', '서울특별시 동대문구', '경기도 화성시', '경기도 고양시 일산동구', '경기도 남양주시', '경기도 고양시 덕양구', '경기도 의정부시', '경기도 안양시 동안구', '경기도 광명시', '울산광역시 남구', '경기도 김포시', '경기도 성남시 수정구', '경기도 안양시 만안구', '경기도 성남시 중원구', '경기도 고양시 일산서구', '세종특별자치시 세종시', '전라북도 전주시 완산구', '울산광역시 울주군', '경기도 하남시',

In [40]:
# 각 지역별로 가장 많은 차종 데이터 찾기
def most_popular_car(region_data):
    df = pd.read_csv("./socar_reservation_triplog.csv")
    region_car_dict= {}
    try:
        for region in region_data:
            condition = (df['region'] == region)
            parsed_data = df.loc[condition]
            car = parsed_data['car_name'].value_counts(ascending=False).index.tolist()
            # print(car)
            region_car_dict[region] = car[0]
    except Exception as e:
        print(f"Error : {e}")
    print(region_car_dict)
    return region_car_dict
result_region_and_car = most_popular_car(region_list)

car = dict()
for k, v in result_region_and_car.items():
    if v in car:
        car[v] += 1
    else:
        car[v] = 1
print(car)

{'서울특별시 강남구': '더뉴레이', '서울특별시 강서구': '아반떼AD', '경기도 성남시 분당구': '코나', '서울특별시 성북구': '아반떼AD', '서울특별시 서대문구': '아반떼AD', '서울특별시 동대문구': '코나', '경기도 화성시': '아반떼AD', '경기도 고양시 일산동구': '코나', '경기도 남양주시': '더뉴레이', '경기도 고양시 덕양구': '아반떼AD', '경기도 의정부시': '아반떼AD', '경기도 안양시 동안구': '아반떼AD', '경기도 광명시': '아반떼AD', '울산광역시 남구': '아반떼AD', '경기도 김포시': '아반떼AD', '경기도 성남시 수정구': '아반떼AD', '경기도 안양시 만안구': '아반떼AD', '경기도 성남시 중원구': '아반떼AD', '경기도 고양시 일산서구': '더뉴레이', '세종특별자치시 세종시': '아반떼AD', '전라북도 전주시 완산구': '아반떼AD', '울산광역시 울주군': '아반떼AD', '경기도 하남시': '코나', '경기도 구리시': '아반떼AD', '전라북도 전주시 덕진구': '레이', '부산광역시 강서구': '아반떼AD', '울산광역시 동구': '아반떼AD', '울산광역시 북구': '아반떼AD', '울산광역시 중구': '아반떼AD', '경기도 양평군': '아반떼AD'}
{'더뉴레이': 3, '아반떼AD': 22, '코나': 4, '레이': 1}


In [3]:

df.describe()

,reservation_id,car_id,member_age,member_total_distance,zone_lat,zone_lng,reservation_created_lat,reservation_created_lng
count,7.515480e+05,751548.000000,751548.000000,738010.000000,751548.000000,751548.000000,199701.000000,199701.000000
mean,2.395360e+07,11032.632738,30.747854,4706.664131,37.286415,127.167624,37.263305,127.134576
std,2.155359e+06,3979.120987,8.424734,6908.222464,0.638582,0.609597,0.880109,1.513531
min,1.960042e+07,4169.000000,21.000000,0.000000,35.083052,126.597065,-42.968955,-123.128056
25%,2.207456e+07,7833.000000,25.000000,750.000000,37.395187,126.883526,37.391814,126.901675
50%,2.396182e+07,10644.000000,28.000000,2302.000000,37.520612,127.041004,37.516770,127.038024
75%,2.584444e+07,14507.000000,35.000000,5923.000000,37.590836,127.122589,37.588336,127.121718
max,2.762850e+07,19449.000000,85.000000,121163.000000,37.760854,129.432836,49.277712,151.179826


In [9]:
# 부름서비스를 2번 이상 사용하는 경우
def vroom_customer():
    df = pd.read_csv("./socar_reservation_triplog.csv")
    condition = (df['is_vroom'] == True)
    vroom_customer = df.loc[condition]
    customer_list = vroom_customer['member_id_encrypted'].value_counts(ascending=False)
    
    print(customer_list)
    
    print("customer : ",customer_list[0])
    # 부름서비스를 2번이상 이용한 고객
    vroom_condition = customer_list.index.tolist()
    
    vroom_twice = []
    for idx, value in enumerate(vroom_condition):
        if customer_list[idx] > 1:
            vroom_twice.append(value)
    
    # print(vroom_twice)
    print(f"vroom twice : {len(vroom_twice)}")

vroom_customer()

8BW1xZn78pBAu+xKWJyP5PjnKlYvAtkbWt/xagMyqso=    64
5h5K83rplUVO9E/ZMOUEVZ/grNxNX7mI7u483FiuLWc=    64
TzUogkpzKt+n7DqRec9ZgD0SU+SHCpA3MMQU3+geVIo=    54
EPWYa4n6zrt72eI1+/aH1rAaXc53S+hWn2omdQ61DSg=    48
CDNoBYU77Xd2KsYGcPef49pC5XvnzhWYX9V2ouZ034s=    48
                                                ..
k1MO21y5JJ1BedZ43/k3QVZ88dKlM+Zrg4vE0CnMias=     1
FYYQxYslnf1VCDq+KxIYluU+JK1RbLeF3Ljk1YzxhXs=     1
wAQVfcHQ/rycpZdjMfYvr6jEe1mStYeQ7mZdYcjJ02U=     1
Wam9THdDysfZrEex04DziQM+qJjwK8PmvDAudVvWp8Q=     1
yTtje1tICB0hOXigTTR7chG+FKywIYJQMRBnN4gfobg=     1
Name: member_id_encrypted, Length: 33348, dtype: int64
customer :  64
vroom twice : 8823


In [5]:
# 유일한 값 별로 갯수 세기
df['member_id_encrypted'].value_counts()
# 1번만 이용한 사람도 많고, 여러번 이용한 사람도 많음

MRo1KLaA/jViPznvtnLB/tHLuvX3XjHsk2dlDkiPfZM=    394
uhvTaMs9pNzgAk4Gvng4J5ZwCbHG4QXFDUNv0zx6OWg=    295
MGKifPpgs6UKyZhi9LEqHKO9MckfSlKq1PPQF+QVHko=    294
6qy5H7fNlSkZ9kKR2gHyTi88JpaOeAGSBiCkqPXQ9GA=    292
q3fh7eiKNMiF5flg008ZJ4E/Bsyf9Q0xmAOq7sm8tDY=    255
                                               ... 
p1lEsLI5w+IJzcNJdokw19u7KrZkQT51LmNR64qfXP0=      1
ThiivMBt6VjBxcP6olEOEHDfp1V61uvZixF59IMLOow=      1
APIjT6NHYFbwSLeBiY6Ok8OGSBwluBvsBmxye32kuAY=      1
GoD5rbN78ZN9QBKbzjvfKRqLVZEXKs8dwZKdnf2QPqA=      1
a/uIPg1p6kr2VDw+JahIxiMfLI5Bu6RxROAgR/ErVtA=      1
Name: member_id_encrypted, Length: 227120, dtype: int64

### 이상치 데이터 없애기
1. 예약 시간 계산해서 없애기 (반납시간 reservation_return_at,	렌트시간 reservation_start_at)

In [6]:
from datetime import datetime
df["reservation_return_at"] = pd.to_datetime(df["reservation_return_at"])
df["reservation_start_at"] = pd.to_datetime(df["reservation_start_at"])
df["total_rent_time"] = (df["reservation_return_at"] - df["reservation_start_at"]).dt.total_seconds()

In [7]:
df['total_rent_time']

0          86622.0
1          72535.0
2           9600.0
3         121989.0
4         108363.0
            ...   
751543     11325.0
751544     82198.0
751545      9576.0
751546     25158.0
751547     15454.0
Name: total_rent_time, Length: 751548, dtype: float64

In [8]:
df = df.sort_values(by='total_rent_time')
df.describe()

,reservation_id,car_id,member_age,member_total_distance,zone_lat,zone_lng,reservation_created_lat,reservation_created_lng,total_rent_time
count,7.515480e+05,751548.000000,751548.000000,738010.000000,751548.000000,751548.000000,199701.000000,199701.000000,7.515480e+05
mean,2.395360e+07,11032.632738,30.747854,4706.664131,37.286415,127.167624,37.263305,127.134576,2.849723e+04
std,2.155359e+06,3979.120987,8.424734,6908.222464,0.638582,0.609597,0.880109,1.513531,4.610510e+04
min,1.960042e+07,4169.000000,21.000000,0.000000,35.083052,126.597065,-42.968955,-123.128056,-5.742000e+05
25%,2.207456e+07,7833.000000,25.000000,750.000000,37.395187,126.883526,37.391814,126.901675,7.467000e+03
50%,2.396182e+07,10644.000000,28.000000,2302.000000,37.520612,127.041004,37.516770,127.038024,1.463100e+04
75%,2.584444e+07,14507.000000,35.000000,5923.000000,37.590836,127.122589,37.588336,127.121718,2.844200e+04
max,2.762850e+07,19449.000000,85.000000,121163.000000,37.760854,129.432836,49.277712,151.179826,4.677842e+06


In [9]:
# 빌린 기간 음수 데이터 삭제
negative_rent_time_index = df[df['total_rent_time'] <=0 ].index
print(negative_rent_time_index)

Int64Index([ 26138, 437097, 196564, 165906, 451018,  71240,  48544,  48346,
            504095, 483364, 195946, 361564,  20753, 407563, 337835, 459578,
            163568, 113618, 399084,  26239, 331036, 201310, 368552, 174110,
            173578, 190789, 413587, 594084, 326875, 703775, 645802, 656829,
            527640, 320576, 410723,  43798, 733441, 488068, 310813, 289192,
            713357, 208112, 411659, 414893, 471126, 264481, 378603, 317114,
            368602, 464162, 467193, 466089, 589668, 561665, 359933, 721007,
            749379, 569890, 422089, 142942, 656973, 297481,  73019, 200754,
            187153, 525668, 260555, 345901, 164220, 183634, 682413,  51370,
            620373, 131061, 509902, 309505, 355842,  38564, 166227],
           dtype='int64')


In [10]:
df = df.drop(negative_rent_time_index)
df = df.sort_values(by='total_rent_time')
df.head()

,reservation_id,car_id,member_id_encrypted,region,reservation_return_at,reservation_start_at,member_age,member_gender,member_created_date,member_total_distance,...,zone_address,zone_lat,zone_lng,zone_type1,zone_type2,zone_type3,reservation_created_lat,reservation_created_lng,trip,total_rent_time
334679,21652995,5452,tAN/axsc8NW0LmEj5yHkYv/XTXJn2zoqkPTqX44RFGQ=,서울특별시 강남구,2019-03-14 21:25:51,2019-03-14 21:20:00,21,female,2019-03-14,643.0,...,서울 강남구 신사동 664-23,37.526798,127.039551,TRANSFER_SUBWAY,COMMERCIAL_OFFICE,COMMERCIAL_HOTSPOT,NaN,NaN,NaN,351.0
661362,27436346,18295,DkqlEifWzyaaEIoGka+i4l8GimXzOiHKPPvax4ej+WY=,경기도 의정부시,2019-11-21 22:27:32,2019-11-21 22:20:00,29,male,2018-09-12,0.0,...,경기 의정부시 의정부동 496-8,37.737343,127.043846,TRANSFER_SUBWAY,COMMERCIAL_HOTSPOT,ETC,37.738644,127.043314,NaN,452.0
663330,27454739,9273,PZ19ME6jFBpRck8IwWWBTK3hhKhhg62BQmZjPDLwbAk=,경기도 성남시 분당구,2019-11-22 19:07:36,2019-11-22 19:00:00,37,male,2017-09-11,30131.0,...,경기 성남시 분당구 야탑동 341,37.412778,127.126776,TRANSFER_TERMINAL,COMMERCIAL_OFFICE,COMMERCIAL_HOTSPOT,NaN,NaN,NaN,456.0
75436,20964994,13424,mHcRghIr1PmdtZvFv7GnMuX4kMGWPVgwMZmEfal0abs=,서울특별시 강서구,2019-02-08 04:57:50,2019-02-08 04:50:00,25,male,2019-01-23,3890.0,...,서울 강서구 방화동 484-17,37.573521,126.810539,LIVING_VILLA,LIVING_ETC,ETC,NaN,NaN,NaN,470.0
32799,20540298,6019,Wn8/pQgM6QOYw4UqxMHAVhG7X/Pm8xKe8UNOvAerpyw=,서울특별시 강남구,2019-01-17 03:28:06,2019-01-17 03:20:00,34,male,2016-01-30,795.0,...,서울 강남구 역삼동 602,37.504848,127.027267,TRANSFER_SUBWAY,COMMERCIAL_HOTSPOT,COMMERCIAL_LODGE,NaN,NaN,NaN,486.0


In [11]:
df.describe()

,reservation_id,car_id,member_age,member_total_distance,zone_lat,zone_lng,reservation_created_lat,reservation_created_lng,total_rent_time
count,7.514690e+05,751469.000000,751469.000000,737932.000000,751469.000000,751469.000000,199689.000000,199689.000000,7.514690e+05
mean,2.395366e+07,11032.628399,30.747435,4706.669514,37.286396,127.167638,37.263294,127.134583,2.850134e+04
std,2.155375e+06,3979.134857,8.424515,6907.426822,0.638602,0.609615,0.880131,1.513576,4.610104e+04
min,1.960042e+07,4169.000000,21.000000,0.000000,35.083052,126.597065,-42.968955,-123.128056,3.510000e+02
25%,2.207466e+07,7833.000000,25.000000,750.000000,37.395187,126.883526,37.391814,126.901675,7.469000e+03
50%,2.396195e+07,10644.000000,28.000000,2302.000000,37.520612,127.041004,37.516768,127.038024,1.463300e+04
75%,2.584451e+07,14507.000000,35.000000,5923.000000,37.590836,127.122589,37.588338,127.121719,2.844500e+04
max,2.762850e+07,19449.000000,85.000000,121163.000000,37.760854,129.432836,49.277712,151.179826,4.677842e+06


2. 운전 거리(member_total_distance)가 0인 것들 삭제

In [12]:
test_data = df.loc[df["member_id_encrypted"] == "uhvTaMs9pNzgAk4Gvng4J5ZwCbHG4QXFDUNv0zx6OWg="]
test_data.head()

,reservation_id,car_id,member_id_encrypted,region,reservation_return_at,reservation_start_at,member_age,member_gender,member_created_date,member_total_distance,...,zone_address,zone_lat,zone_lng,zone_type1,zone_type2,zone_type3,reservation_created_lat,reservation_created_lng,trip,total_rent_time
482932,24107727,9252,uhvTaMs9pNzgAk4Gvng4J5ZwCbHG4QXFDUNv0zx6OWg=,서울특별시 성북구,2019-07-01 22:20:00,2019-07-01 22:10:00,22,male,2019-01-15,32039.0,...,서울 성북구 삼선동4가 18-6,37.588623,127.010368,LIVING_ETC,COMMERCIAL_OFFICE,ETC,NaN,NaN,NaN,600.0
327274,21580456,9252,uhvTaMs9pNzgAk4Gvng4J5ZwCbHG4QXFDUNv0zx6OWg=,서울특별시 성북구,2019-03-10 20:00:00,2019-03-10 19:50:00,21,male,2019-01-15,32039.0,...,서울 성북구 삼선동4가 18-6,37.588623,127.010368,LIVING_ETC,COMMERCIAL_OFFICE,ETC,NaN,NaN,NaN,600.0
542798,26336173,9252,uhvTaMs9pNzgAk4Gvng4J5ZwCbHG4QXFDUNv0zx6OWg=,서울특별시 성북구,2019-10-02 22:00:27,2019-10-02 21:50:00,22,male,2019-01-15,32039.0,...,서울 성북구 삼선동4가 18-6,37.588623,127.010368,LIVING_ETC,COMMERCIAL_OFFICE,ETC,37.587779,127.008610,NaN,627.0
556930,26467768,9567,uhvTaMs9pNzgAk4Gvng4J5ZwCbHG4QXFDUNv0zx6OWg=,서울특별시 성북구,2019-10-08 15:50:59,2019-10-08 15:40:00,22,male,2019-01-15,32039.0,...,서울 성북구 삼선동4가 18-6,37.588623,127.010368,LIVING_ETC,COMMERCIAL_OFFICE,ETC,37.587002,127.009505,NaN,659.0
533956,26251297,16424,uhvTaMs9pNzgAk4Gvng4J5ZwCbHG4QXFDUNv0zx6OWg=,서울특별시 성북구,2019-09-29 03:45:56,2019-09-29 03:30:00,22,male,2019-01-15,32039.0,...,서울 성북구 삼선동4가 18-6,37.588623,127.010368,LIVING_ETC,COMMERCIAL_OFFICE,ETC,37.588508,127.010274,NaN,956.0


In [13]:
test_data['car_name']

482932     투싼(경유)
327274     투싼(경유)
542798     투싼(경유)
556930       스포티지
533956         코나
           ...   
62192          코나
410632     투싼(경유)
689612    티볼리(경유)
239438         코나
201730         코나
Name: car_name, Length: 295, dtype: object

In [41]:
test_data.head()
test_data["member_total_distance"]

482932    32039.0
327274    32039.0
542798    32039.0
556930    32039.0
533956    32039.0
           ...   
62192     32039.0
410632    32039.0
689612    32039.0
239438    32039.0
201730    32039.0
Name: member_total_distance, Length: 295, dtype: float64

- 겹치는 id를 가진 고객이라도 member_total_distance는 동일
  - 즉, 이용 즉시 갱신되어서 모두 같은 값을 가지고 있다는 것
  - 그러므로 member_total_distance가 0인 고객들은 처음 사용한 고객 혹은 실질적으로 운전하지 않은 사람들의 데이터라고 볼 수 있음
  - 보통 거리가 시간이 아주 짧음
- 1회당 주행한 거리를 나타내는 지표는 현재 없음

In [14]:
zero_distance = df.loc[df["member_total_distance"] == 0]
zero_distance.sort_values(by='reservation_start_at')

,reservation_id,car_id,member_id_encrypted,region,reservation_return_at,reservation_start_at,member_age,member_gender,member_created_date,member_total_distance,...,zone_address,zone_lat,zone_lng,zone_type1,zone_type2,zone_type3,reservation_created_lat,reservation_created_lng,trip,total_rent_time
8570,20305084,14800,AP5XshJiQ7oASUNHRhDvE3YLimgdGX15e4HUPCp8QlM=,경기도 안양시 만안구,2019-01-04 15:50:20,2019-01-04 12:50:00,47,male,2019-01-04,0.0,...,경기 안양시 만안구 석수동 859-10,37.419422,126.890182,TRANSFER_STATION,COMMERCIAL_OFFICE,TRANSFER_SUBWAY,NaN,NaN,NaN,10820.0
11658,20333861,6382,hYEXuNZfc68it2z+PgRWk+FLtBpM0R4/WNjnFIIxMZo=,경기도 화성시,2019-01-06 01:17:17,2019-01-06 01:00:00,26,female,2019-01-05,0.0,...,경기 화성시 향남읍 행정리 486-4,37.130116,126.922966,LIVING_APT,TRANSFER_TERMINAL,COMMERCIAL_HOTSPOT,NaN,NaN,NaN,1037.0
13244,20348485,9024,WNNPJXCKyr+f6v0XZ6qRwEon3k6og8AeQuQTk06XWPw=,서울특별시 강서구,2019-01-06 18:01:06,2019-01-06 17:50:00,25,male,2017-12-05,0.0,...,서울 강서구 방화2동 612-18,37.563545,126.809723,LIVING_ETC,TRANSFER_SUBWAY,ETC,NaN,NaN,NaN,666.0
13842,20354324,16128,kWFtD/sqSwAU2ETOW0KsPXNjRy/jMAsP+AdL2zmJbk4=,경기도 고양시 일산동구,2019-01-06 22:50:31,2019-01-06 22:30:00,30,female,2019-01-06,0.0,...,경기 고양시 일산동구 장항동 769,37.659439,126.769590,COMMERCIAL_HOTSPOT,TRANSFER_SUBWAY,ETC,NaN,NaN,NaN,1231.0
15026,20366775,9173,03htcmB0sQLITGqWbXu4QfmIy6nj6polsO+tks81Wco=,경기도 고양시 일산서구,2019-01-07 22:42:04,2019-01-07 18:50:00,42,female,2018-02-01,0.0,...,경기 고양시 일산서구 일산동 655-95,37.682220,126.771095,LIVING_APT,TRANSFER_SUBWAY,ETC,NaN,NaN,NaN,13924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677456,27590698,8044,YQeFxLaJzDgOK0Irqno7XxQY5yOLa8XqdsPEJNrgnLI=,부산광역시 강서구,2019-11-29 11:11:14,2019-11-29 10:50:00,27,male,2017-01-17,0.0,...,부산 강서구 대저2동 2108-1,35.179012,128.951907,TRANSFER_STATION,TRANSFER_SUBWAY,ETC,35.171030,128.949250,NaN,1274.0
678721,27605360,15854,FWMhTqUP8exAPOnYysjtDaFRlKQA+tnyY+DjJA/8Ick=,전라북도 전주시 덕진구,2019-11-30 13:47:05,2019-11-30 13:30:00,23,female,2018-07-06,0.0,...,전북 전주시 덕진구 우아동3가 752-59,35.848291,127.158993,TRANSFER_STATION,COMMERCIAL_HOTSPOT,NaN,NaN,NaN,NaN,1025.0
678148,27598585,11153,jq83o9VRsuTH0+fwNNB/Mp7mikAa6Y43a+3PfZBfOU0=,서울특별시 강남구,2019-11-30 14:57:34,2019-11-30 13:40:00,36,male,2019-11-29,0.0,...,서울 강남구 논현동 122-8,37.510719,127.023392,TRANSFER_SUBWAY,COMMERCIAL_OFFICE,COMMERCIAL_HOTSPOT,37.509400,127.055059,서울특별시 강남구,4654.0
679678,27617053,8990,A6Iy1OTiPosQp6zLQFg+4TL5WgKDA7gOMx2ayl3Pfcc=,서울특별시 강서구,2019-11-30 14:30:08,2019-11-30 14:20:00,37,female,2019-11-11,0.0,...,서울 강서구 화곡동 1045,37.541691,126.834740,LIVING_VILLA,LIVING_VILLA,LIVING_ETC,37.530542,126.852065,NaN,608.0


#### 움직이지 않아도 triplog가 작성된걸로 보아 일정시간마다 자동으로 작성 된 듯

In [44]:
# 움직인 거리가 없는 데이터 들은 삭제
df = df.drop(zero_distance.index)
df.describe()

,reservation_id,car_id,member_age,member_total_distance,zone_lat,zone_lng,reservation_created_lat,reservation_created_lng,total_rent_time
count,7.511390e+05,751139.000000,751139.000000,737602.000000,751139.000000,751139.000000,199604.000000,199604.000000,7.511390e+05
mean,2.395367e+07,11032.791258,30.746597,4708.775258,37.286384,127.167651,37.263288,127.134560,2.851019e+04
std,2.155378e+06,3979.199484,8.424358,6908.254210,0.638615,0.609632,0.880214,1.513830,4.610707e+04
min,1.960042e+07,4169.000000,21.000000,1.000000,35.083052,126.597065,-42.968955,-123.128056,3.510000e+02
25%,2.207471e+07,7833.000000,25.000000,752.000000,37.395187,126.883526,37.391815,126.901702,7.477000e+03
50%,2.396196e+07,10645.000000,28.000000,2304.000000,37.520612,127.041004,37.516765,127.038026,1.463900e+04
75%,2.584455e+07,14507.000000,35.000000,5924.750000,37.590836,127.122589,37.588334,127.121726,2.845550e+04
max,2.762850e+07,19449.000000,85.000000,121163.000000,37.760854,129.432836,49.277712,151.179826,4.677842e+06


### 불필요한 컬럼 삭제

In [48]:
for col in df.columns:
    print(col)

reservation_id
car_id
member_id_encrypted
region
reservation_return_at
reservation_start_at
member_age
member_gender
member_created_date
member_total_distance
is_vroom
car_name
zone_name
zone_address
zone_lat
zone_lng
zone_type1
zone_type2
zone_type3
reservation_created_lat
reservation_created_lng
trip
total_rent_time


- 예약한 곳의 위도나 경도는 크게 중요하지 않을 듯 그러므로 삭제해도 될 듯
- 차량 사용 시작시간(reservation_start_at)과 예약완료 시간이 있었다면 오히려 사용성이 높았을듯
  - 예를들면, 급하게 예약하고 사용했다는 것이니까 위치가 중요할듯 (행사가 있는 곳, 혹은 유동인구가 많은곳으로 추정할 수 있음, 또한 연령대와 같이 비교해서 어떤 연령이 예약직후 바로 사용하는지도 확인 가능 할 듯)

In [ ]:
df = df.drop(columns=["reservation_created_lat", "reservation_created_lng"])

In [51]:
df.describe()

,reservation_id,car_id,member_age,member_total_distance,zone_lat,zone_lng,total_rent_time
count,7.511390e+05,751139.000000,751139.000000,737602.000000,751139.000000,751139.000000,7.511390e+05
mean,2.395367e+07,11032.791258,30.746597,4708.775258,37.286384,127.167651,2.851019e+04
std,2.155378e+06,3979.199484,8.424358,6908.254210,0.638615,0.609632,4.610707e+04
min,1.960042e+07,4169.000000,21.000000,1.000000,35.083052,126.597065,3.510000e+02
25%,2.207471e+07,7833.000000,25.000000,752.000000,37.395187,126.883526,7.477000e+03
50%,2.396196e+07,10645.000000,28.000000,2304.000000,37.520612,127.041004,1.463900e+04
75%,2.584455e+07,14507.000000,35.000000,5924.750000,37.590836,127.122589,2.845550e+04
max,2.762850e+07,19449.000000,85.000000,121163.000000,37.760854,129.432836,4.677842e+06
